## Revisión datos del banco mundial, sobre la poblacion en 2023

In [1]:
# librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

Usando información sobre cantidades de población de "https://es.wikipedia.org/wiki/Poblaci%C3%B3n_humana_mundial#Pa%C3%ADses_y_territorios_dependientes_por_poblaci%C3%B3n_2024", se buscara la población de Chile.

In [3]:
url = "https://es.wikipedia.org/wiki/Poblaci%C3%B3n_humana_mundial#Pa%C3%ADses_y_territorios_dependientes_por_poblaci%C3%B3n_2024"
page = requests.get(url).text
soup = bs(page)
tables = soup.find_all('table')
len(tables)

22

In [4]:
tables[3]

<table class="wikitable">
<tbody><tr>
<th>Puesto
</th>
<th>País o territorio dependiente
</th>
<th>Población <br/> (proyección 2024)<sup class="reference separada" id="cite_ref-:22_16-0"><a href="#cite_note-:22-16"><span class="corchete-llamada">[</span>15<span class="corchete-llamada">]</span></a></sup>​
</th></tr>
<tr>
<td>1.º
</td>
<td>India  
</td>
<td>1 450 935 779  
</td></tr>
<tr>
<td>2.º
</td>
<td>China  
</td>
<td>1 419 321 279  
</td></tr>
<tr>
<td>3.º
</td>
<td>Estados Unidos  
</td>
<td>345 426 567  
</td></tr>
<tr>
<td>4.º
</td>
<td>Indonesia  
</td>
<td>283 487 932  
</td></tr>
<tr>
<td>5.º
</td>
<td>Pakistán  
</td>
<td>251 269 158  
</td></tr>
<tr>
<td>6.º
</td>
<td>Nigeria  
</td>
<td>232 679 482  
</td></tr>
<tr>
<td>7.º
</td>
<td>Brasil  
</td>
<td>211 998 564  
</td></tr>
<tr>
<td>8.º
</td>
<td>Bangladés  
</td>
<td>173 562 367  
</td></tr>
<tr>
<td>9.º
</td>
<td>Rusia  
</td>
<td>144 820 423  
</td></tr>
<tr>
<td>10.º
</td>
<td>Etiopía  
</td>
<td>132 059 770  
</t

In [5]:
#creamos un DataFrame vacío con los títulos de la tabla
df = pd.DataFrame(columns = ["Pais o territorios dependientes",'Población proyectada a 2024'])

# iterar sobre cada fila ('tr') para completar la información
for row in tables[3].find_all('tr')[1::]:
    cols = row.find_all("td")
    #print(cols)
    cols = [col.text.strip() for col in cols]
    #print(cols)
    country = cols[1]
    population = int(cols[2].replace(" ", ""))
    new_row = pd.DataFrame({'Pais o territorios dependientes': country, 'Población proyectada a 2024': population}, index=['Pais o territorios dependientes'])
    df = pd.concat([df, new_row], ignore_index=True)
df

,Pais o territorios dependientes,Población proyectada a 2024
0,India,1450935779
1,China,1419321279
2,Estados Unidos,345426567
3,Indonesia,283487932
4,Pakistán,251269158
...,...,...
232,Montserrat,4414
233,Islas Malvinas,3492
234,Tokelau,2528
235,Niue,1841


En base a la información en wikipedia, crea una dataframe con los paises reconocidos y su población estimada para el 2024.

In [7]:
df[df["Pais o territorios dependientes"] == "Chile"]

,Pais o territorios dependientes,Población proyectada a 2024
65,Chile,19764772


In [8]:
lista_indice = df.index[df["Pais o territorios dependientes"] == "Chile"].to_list()
indice = lista_indice[0]
poblacion = df.at[indice,"Población proyectada a 2024"]
poblacion

19764772

Se busca la población proyectada para el 2024 de Chile, en base a la cual se buscará países con poblaciones en el intervalo [16.000.000, 24.000.000] más o menos que la población de Chile.

In [11]:
paises_rango = df.loc[(df["Población proyectada a 2024"] >= 16000000) & (df["Población proyectada a 2024"] <= 24000000)]
paises_rango

,Pais o territorios dependientes,Población proyectada a 2024
58,Burkina Faso,23548782
59,Taiwán,23213963
60,Sri Lanka,23103559
61,Malaui,21655290
62,Zambia,21314960
63,Kazajistán,20592575
64,Chad,20299130
65,Chile,19764772
66,Rumania,19015096
67,Somalia,19009156


Para este analisis intentaremos ocupar datos de paises de Africa, Asia, Europa y otro país de America, que tengan un modelo de open data sobre licitaciones y compras del estado, que seran comparados con datos de Chile.

## Revisión de datos del csv de la ocds.

Usando el buscador de OpenContracting se buscaran datos sobre Chile y sobre los Paises en el DataFrame antes visto. (buscador: https://data.open-contracting.org/es/search/). Se preferirá los países que estan más cerca de la población de Chile y que tengan información en Open Contracting, si hay algún continente que no tenga un país en el rango y que tenga datos en Open Contracting, se encontrara un país del continente y se mantendra solo si la cantidad de población estimada existe entre [10.000.000 , 30.000.000].

Se buscó información en el registro de datos de Open Contracting y se observó que no existe información sobre países de Asia en el rango buscado, en Africa se encontraron datos sobre Zambia, en Europa sobre Romania y en America sobre Ecuador, puesto que Guatemala no tenia datos en el rango de años querido.

Se revisarán los archivos que existen en cada grupo de archivos descargado y se usaran, solo los que existen en todos.

In [23]:
%cd Zambia
%cd 2018
%ls
%cd ..
%cd 2021
%ls
%cd ..
%cd ..

C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Zambia
C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Zambia\2018
 El volumen de la unidad C es OS
 El número de serie del volumen es: 240D-CD63

 Directorio de C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Zambia\2018

23-08-2024  21:50    <DIR>          .
06-10-2024  18:56    <DIR>          ..
23-08-2024  21:50            29.352 awards.csv
23-08-2024  21:50            24.526 awards_items.csv
23-08-2024  21:50            16.160 awards_suppliers.csv
23-08-2024  21:50            29.663 contracts.csv
23-08-2024  21:50            24.665 contracts_items.csv
23-08-2024  21:50           307.593 main.csv
23-08-2024  21:50           263.155 parties.csv
23-08-2024  21:50           142.803 tender_documents.csv
23-08-2024  21:50            47.002 tender_items.csv
23-08-2024  21:50            49.425 tender_tenderers.csv
              10 archivos        934.344 bytes
               2 dirs  303.112.478.720 byte

In [25]:
%cd Romania
%cd 2018
%ls
%cd ..
%cd 2021
%ls
%cd ..
%cd ..

C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Romania
C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Romania\2018
 El volumen de la unidad C es OS
 El número de serie del volumen es: 240D-CD63

 Directorio de C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Romania\2018

04-07-2024  14:16    <DIR>          .
06-10-2024  18:57    <DIR>          ..
04-07-2024  14:16         4.392.832 awards.csv
04-07-2024  14:16        13.140.479 awards_documents.csv
04-07-2024  14:16         2.709.386 awards_items.csv
04-07-2024  14:16        10.473.146 awards_suppliers.csv
04-07-2024  14:16         3.855.379 bids_details.csv
04-07-2024  14:16        11.285.552 bids_details_tenderers.csv
04-07-2024  14:16         4.811.646 bids_statistics.csv
04-07-2024  14:16        38.818.890 main.csv
04-07-2024  14:16        18.201.247 parties.csv
04-07-2024  14:16         6.757.153 parties_additionalIdentifiers.csv
04-07-2024  14:16         3.759.498 tender_documents.csv
04-0

In [27]:
%cd Chile
%cd 2018
%ls
%cd ..
%cd 2021
%ls
%cd ..
%cd ..

C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Chile
C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Chile\2018
 El volumen de la unidad C es OS
 El número de serie del volumen es: 240D-CD63

 Directorio de C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Chile\2018

06-10-2024  13:35    <DIR>          .
06-10-2024  18:57    <DIR>          ..
06-10-2024  21:01    <DIR>          .ipynb_checkpoints
19-08-2024  11:45        64.571.337 awards.csv
19-08-2024  11:45       137.878.898 awards_documents.csv
19-08-2024  11:45       172.974.592 awards_items.csv
19-08-2024  11:45        30.901.082 awards_suppliers.csv
19-08-2024  11:45       165.869.422 main.csv
19-08-2024  11:45       370.417.597 parties.csv
19-08-2024  11:45       243.504.970 parties_additionalIdentifiers.csv
19-08-2024  11:45       287.507.728 tender_items.csv
19-08-2024  11:45       103.927.462 tender_tenderers.csv
               9 archivos  1.577.553.088 bytes
               3 dirs  303.

In [29]:
%cd Ecuador
%cd 2018
%ls
%cd ..
%cd 2021
%ls
%cd ..
%cd ..

C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Ecuador
C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Ecuador\2018
 El volumen de la unidad C es OS
 El número de serie del volumen es: 240D-CD63

 Directorio de C:\Users\camil\OneDrive\Escritorio\Proyecto ciencia de datos\Ecuador\2018

04-10-2024  01:24    <DIR>          .
06-10-2024  18:57    <DIR>          ..
04-10-2024  01:24         3.263.178 auctions.csv
04-10-2024  01:24         2.731.412 auctions_stages.csv
04-10-2024  01:24        36.276.594 auctions_stages_bids.csv
04-10-2024  01:24        32.972.588 auctions_stages_bids_tenderers.csv
04-10-2024  01:24         4.482.324 auctions_stages_rounds.csv
04-10-2024  01:24         4.169.593 auctions_winningBids.csv
04-10-2024  01:24         3.661.724 auctions_winningBids_tenderers.csv
04-10-2024  01:24        34.021.354 awards.csv
04-10-2024  01:24        73.214.373 awards_items.csv
04-10-2024  01:24        84.848.700 awards_items_additionalClassifications.csv

Se descargo desde la pagina https://data.open-contracting.org/es/publication/10#access el archivo en formato csv del 2018. Se unirá la información a un dataframe usando la columna _link en el archivo main y _link_main en los otros dataframes.

In [33]:
from os import path
import pandas as pd

lista_doc = ["main.csv","awards.csv", "awards_items.csv", "awards_suppliers.csv", "tender_items.csv", "tender_tenderers.csv"]
contador = 0
for documento in lista_doc:
    print(documento)
    camino = path.join( "Chile","2018", documento)
    if contador == 0:
        df_main = pd.read_csv(camino, delimiter=",")
        
    elif contador == 1:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas = list(df_e.columns.values)
        
        if "description" in lista_columnas :
            df_e.rename(columns={'description': f'description_{documento}'}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
        df_main = df_main.merge(df_e, left_on="_link", right_on="_link_main")
        
    else:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas =  list(df_e.columns.values)
        if "description" in lista_columnas:
            df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}'}, inplace=True)
        df_main = df_main.merge(df_e, on="_link_main")
    contador += 1
    
df_main

main.csv
awards.csv
awards_items.csv
awards_suppliers.csv
tender_items.csv


KeyboardInterrupt: 

In [ ]:
df_main.info()

In [ ]:
df_main.describe()

In [72]:

contador = 0
for documento in lista_doc:
    print(documento)
    camino = path.join( "Chile","2021", documento)
    if contador == 0:
        df_ma = pd.read_csv(camino, delimiter=",")
        
    elif contador == 1:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas = list(df_e.columns.values)
        
        if "description" in lista_columnas :
            df_e.rename(columns={'description': f'description_{documento}'}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
        df_ma = df_ma.merge(df_e, left_on="_link", right_on="_link_main")
        
    else:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas =  list(df_e.columns.values)
        if "description" in lista_columnas:
            df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}'}, inplace=True)
        df_ma = df_ma.merge(df_e, on="_link_main")
    contador += 1

df_ma

main.csv


C:\Users\camil\AppData\Local\Temp\ipykernel_1676\79629777.py:6: DtypeWarning: Columns (34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ma = pd.read_csv(camino, delimiter=",")


awards.csv
awards_items.csv
awards_suppliers.csv
parties.csv


MemoryError: Unable to allocate 15.4 GiB for an array with shape (48, 43177742) and data type object